OneHotEncoder を用いてカテゴリ変数をエンコーディングする

Kaggleの Housing Prices Competitions for Kaggle learn users のデータを使用して
OneHotEncoder の使い方を学んでいきます。カテゴリ変数とは、その要素が数値ではなく
文字列であるような要素のことを言います（例えばアンケート結果の「良い」「普通」「悪い」といった
要素が格納されているカラムのことをカテゴリ変数と言います）。このままではモデルに変数を渡す
ことができないため、何らかの法則に則り数値変数に変換する必要があります。その手法の一つが
OneHotEncodingです。

OneHotEncodingはそのカテゴリの水準に属するか否かを二値変数（0,1）で返し、それを要素とす
る新たなカラムを足していくハンドリング手法になります。したがって、水準がn個存在するとき新たに
n列増えることになります。これを全てのカテゴリ変数を持つカラムに対して行うのでデータが大きく
なりやすいという欠点があることに注意してください。

OneHotEncodingはpandasのget_dummies関数を用いる方法もありますが、ここでは
scikit-learnのpreprocessingモジュールのOneHotEncoderを用いることにします。

まずはベースラインモデルを構築してスコアを確認してみます。

In [1]:
# ここではId列で添字付けられた各住宅に関する80の特徴量（床面積、車庫の有無、築年数など）
# が格納されたテーブルデータを用います。このコンペではこれらの特徴量を用いて各住宅の住宅価格を推定することが目的となります。
# 使用するデータのshapeは訓練データ(1460 x 81)、テストデータ(1459 x 80)となります。target変数は住宅価格である SalePrice です。
# 訓練データをtrain、テストデータをtestとして読み込みます。
import pandas as pd

train = pd.read_csv('~/Downloads/kaggle_house/train.csv')
test = pd.read_csv('~/Downloads/kaggle_house/test.csv')

In [2]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
# yに目的変数であるSalePriceを、Xに特徴量を格納します。

y = train.SalePrice
X = train.drop(['SalePrice'], axis=1)

# カテゴリ変数のコラム名を取得します。

cat_cols = [cname for cname in X.columns if X[cname].dtype == 'object']

# X, y を訓練データとバリデーションデータに分けます。

from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# 前処理を施していきます。数値変数の欠損値はSimpleImputerを用いて中央値で補完し、
# カテゴリ変数の欠損値補完には最頻値を、エンコーディングにはOneHotEncoderを用います。

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

imputed_train_X_cat = pd.DataFrame(cat_imputer.fit_transform(train_X[cat_cols]))
imputed_val_X_cat = pd.DataFrame(cat_imputer.transform(val_X[cat_cols]))

#抜け落ちたカラム名をもう一度付け直す必要があります。
imputed_train_X_cat.columns = train_X[cat_cols].columns
imputed_val_X_cat.columns = val_X[cat_cols].columns

In [5]:
# OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

#SimpleImputer同様DataFrameに直す必要があります。
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(imputed_train_X_cat))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(imputed_val_X_cat))

#imputeではカラムが抜け落ちましたが、OH_encodingではindexが抜け落ちることに注意してください。
OH_cols_train.index = imputed_train_X_cat.index
OH_cols_valid.index = imputed_val_X_cat.index

In [6]:
#数値変数の欠損値補完
num_X_train = train_X.drop(cat_cols, axis=1)
num_X_valid = val_X.drop(cat_cols, axis=1)

imputed_num_X_train = pd.DataFrame(num_imputer.fit_transform(num_X_train))
imputed_num_X_valid = pd.DataFrame(num_imputer.transform(num_X_valid))

imputed_num_X_train.columns = num_X_train.columns
imputed_num_X_valid.columns = num_X_valid.columns

In [7]:
#前処理済みの数値変数とカテゴリ変数を結合します。これでOneHotEncodingを用いた前処理の完成です。
OH_X_train = pd.concat([imputed_num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([imputed_num_X_valid, OH_cols_valid], axis=1)

In [8]:
# モデルの定義です。今回はxgboostを使用することにします。
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.04, n_jobs=4).fit(OH_X_train, train_y)

In [9]:
# 試しにスコア(mean absolute error)を見てみます。

from sklearn.metrics import mean_absolute_error

predictions = xgb_model.predict(OH_X_valid)
score = mean_absolute_error(predictions, val_y)

print(score)

14110.516823630138
